<a href="https://colab.research.google.com/github/KhurramDevOps/Quarter-02/blob/master/05_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploring GEMINI 2.0 Video and Audio Analysis :**

In [2]:
!pip install --upgrade --quiet google-genai

In [3]:
!pip install gTTS --quiet

In [4]:
!pip install playsound --quiet

  Preparing metadata (setup.py) ... done


In [5]:
from google.colab import userdata
Video_process_API:str = userdata.get('GOOGLE_API_KEY')
if(Video_process_API):
  print("Api key found")
else:
  print("Api key not found")

Api key found


In [6]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key = Video_process_API)
model: str = "gemini-2.0-flash-exp"


# **2. Uploading file path and converting in to Readable Format**

In [13]:
# This appears to be a fle path and not python code
video_file_name = "/content/IMG_5457.mp4"

In [14]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
    print("Waiting for video to be processed.")
    time.sleep(10)
    video_file = client.files.get(name= video_file.name or "")

  if video_file.state == "FAILED":
    raise Exception(f"Video processing failed: {video_file.error}")
  print(f"Video processing complete : " + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete : https://generativelanguage.googleapis.com/v1beta/files/ur3rfv39691c


# **3. Giving video to LLM for scenes Descriptions**

In [15]:
from google.genai.types import Content, Part
prompt = """
for each scene in this video,
Generate captions that describe the scene along with any spoken text placed in quotation marks.
Place each caption into an object with the timecode of the caption in the video.
"""

video = my_video

response = client.models.generate_content(
    model= model,
    contents=[
        Content(
            role="User",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
      prompt,
    ]
)
output_text = response.text
print(output_text)


```json
[
  {
    "timecode": "00:00",
    "caption": "A person with dark hair and a dark mustache is lying down and looking at the camera, wearing a black beanie and a blue plaid shirt."
  },
  {
    "timecode":"00:00",
    "caption":"“Hello everyone, my name is Mohammad Khurram and I am from Lahore and learning Agentic AI from PICS Lahore nowadays”."
   }
]
```



# **4. Converting GEMINI text response to Audio using gTTS python library**

In [17]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio, display , Markdown

tts = gTTS(text="My name is Khurram and im from lahore and im learning Agentic AI Now a days from PIAIC ", lang='en')


with open("output.mp3","wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

# Play the audio using IPython.display Audio to ensure it works in colab
# This is more reliable than playsound in Colab environments.

display(Audio("output.mp3",autoplay=True))


# **6. Chatting with Gemini on the basis on video**

In [18]:
from google.genai.chats import Chat

chat: Chat = client.chats.create(
    model=model)

response = chat.send_message(output_text)
Markdown(response.text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A man with dark hair and a mustache, wearing a black beanie and a blue plaid shirt, is lying down and looking directly at the camera."
  },
  {
    "timecode":"00:00",
    "caption":"The man introduces himself as Mohammad Khurram from Lahore, stating that he is currently learning Agentic AI at PICS Lahore."
   }
]
```


In [19]:
response = chat.send_message("What is my person name in this video?")
Markdown(response.text)

Based on the provided captions, your name is **Mohammad Khurram**.


In [20]:
response = chat.send_message("describe whats the person do in key points?")
Markdown(response.text)

Okay, here are the key points describing what the person, Mohammad Khurram, is doing in the video based on the captions:

*   **Lying Down:** He is positioned lying down, not standing or sitting.
*   **Looking at Camera:** He is directly facing and looking at the camera.
*   **Introduction:** He is introducing himself.
*   **Identification:** He states his name is Mohammad Khurram.
*  **Location:** He mentions he's from Lahore.
*   **Learning:** He is currently learning Agentic AI at PICS Lahore.

In summary, Mohammad Khurram is lying down, addressing the camera to introduce himself, state his location, and mention his current studies in Agentic AI.
